# Model Building

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
from typing import List

Set a predetermined seed so all our results can be replicated

In [3]:
RANDOM_SEED = 1337

# Preprocessing

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
full_data = pd.read_csv('Dataset/clean_data.csv', index_col='Customer_ID')

In [24]:
#Separate target column with rest of the data
churn_col = full_data['churn'].copy()
full_data = full_data.drop('churn',axis=1)

In [ ]:
#Separate data for training and testing with 80% for training and 20% testing
#Uses our preselected random seed to results are reproducible 
x_train, x_test, y_train, y_test = train_test_split(
    full_data,
    churn_col,
    test_size=0.2,
    random_state=RANDOM_SEED
)

In [36]:
with open('columnDescriptions.json','r') as f:
    col_desc = json.load(f)
    
#Shortened descriptions with elipses for plot titles
#Only retains first 20 characters of description then appends with elipses
short_col_desc = dict(zip(
    col_desc.keys(),
    map(lambda desc: 
        desc if len(desc)<20 else f'{desc[:20]}...', col_desc.values()
    )
))

We will use several different algorithms then compare their performance afterwards to determine which is the best to use. The algorithm we will use are: 
- Logistic Regression
- K Nearest Neighbor Classifier
- Random Forest
- XGBoost
- LightGBM

In [30]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [38]:
class PipelineFactory:
    
    def impute_ohe_scale(cat_cols:List[str],num_cols:List[str])->Pipeline:
        pipe = make_pipeline(
            SimpleImputer(strategy='most_frequent'),
            Column
            StandardScaler()
        )
    
    def impute_ohe(cat_cols:List[str],num_cols:List[str])->Pipeline:
        return
    
    def ohe(cat_cols:List[str],num_cols:List[str])->Pipeline:
        return
    


In [49]:
type(pipe)

sklearn.pipeline.Pipeline

In [39]:
pipe.fit_transform(full_data)

ValueError: could not convert string to float: 'NORTHWEST/ROCKY MOUNTAIN AREA'

In [33]:
full_data[full_data.columns[full_data.isna().any()]]

,adults,dwlltype,hnd_webcap,income,infobase,lor,new_cell,ownrent,prizm_social_one
Customer_ID,,,,,,,,,
1000001,1.0,0.0,WCMB,4.0,1.0,15.0,NaN,1.0,suburb
1000002,1.0,0.0,WC,5.0,1.0,1.0,0.0,NaN,urban
1000003,2.0,0.0,NaN,5.0,1.0,7.0,1.0,1.0,suburb
1000004,4.0,1.0,NaN,6.0,1.0,6.0,1.0,NaN,town
1000005,1.0,1.0,WCMB,6.0,1.0,5.0,1.0,0.0,urban
...,...,...,...,...,...,...,...,...,...
1099996,5.0,0.0,WC,4.0,1.0,15.0,NaN,1.0,urban
1099997,3.0,0.0,WC,7.0,1.0,9.0,NaN,1.0,suburb
1099998,1.0,NaN,WCMB,9.0,1.0,NaN,NaN,NaN,urban
